In [ ]:
from torch import nn
from torch.quantization import quantize_dynamic
from torchvision.io import read_image 
from ultralytics import YOLO
import torch

import os
import copy

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/ubuntu/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [85]:
img_path = '/container_root/ws/notebooks/data/bus.jpg'
model = YOLO('/container_root/ws/models/yolo11n-seg.pt')
model.export(format='onnx')

# model.info(detailed=True)
# model(img_path)
# print(model)
# img = read_image(img_path)

Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.6.0+cpu CPU (AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx)
YOLO11n-seg summary (fused): 265 layers, 2,868,664 parameters, 0 gradients, 10.4 GFLOPs

PyTorch: starting from '/container_root/ws/models/yolo11n-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (5.9 MB)

ONNX: starting export with onnx 1.16.1 opset 19...
ONNX: slimming with onnxslim 0.1.49...
ONNX: export success ✅ 3.3s, saved as '/container_root/ws/models/yolo11n-seg.onnx' (11.2 MB)

Export complete (4.2s)
Results saved to /container_root/ws/models
Predict:         yolo predict task=segment model=/container_root/ws/models/yolo11n-seg.onnx imgsz=640  
Validate:        yolo val task=segment model=/container_root/ws/models/yolo11n-seg.onnx imgsz=640 data=/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


'/container_root/ws/models/yolo11n-seg.onnx'

# Modules fulsion
Currently torch.quantization.fuse_modules only fuses the following sequence of modules:
- conv, bn
- conv, bn, relu
- conv, relu
- linear, relu
- bn, relu

In [84]:
model = YOLO('/container_root/ws/models/yolo11n-seg.pt')
# model.eval()
model.fuse()
model.export(format='onnx')
# print(model)


YOLO11n-seg summary (fused): 265 layers, 2,868,664 parameters, 0 gradients, 10.4 GFLOPs
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.6.0+cpu CPU (AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx)

PyTorch: starting from '/container_root/ws/models/yolo11n-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (5.9 MB)

ONNX: starting export with onnx 1.16.1 opset 19...
ONNX: slimming with onnxslim 0.1.49...
ONNX: export success ✅ 3.4s, saved as '/container_root/ws/models/yolo11n-seg.onnx' (11.2 MB)

Export complete (3.9s)
Results saved to /container_root/ws/models
Predict:         yolo predict task=segment model=/container_root/ws/models/yolo11n-seg.onnx imgsz=640  
Validate:        yolo val task=segment model=/container_root/ws/models/yolo11n-seg.onnx imgsz=640 data=/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


'/container_root/ws/models/yolo11n-seg.onnx'

# Post-Training Static Quantization (PTQ)

In [ ]:
model = YOLO('/container_root/ws/models/yolo11n-seg.pt')


TypeError: Value after * must be an iterable, not SegmentationModel

In [87]:
# model_quantized = quantize_dynamic(model=model_fused, qconfig_spec={nn.LSTM, nn.Linear}, dtype=torch.qint8, inplace=False)
# model_quantized(img_path)
model = YOLO('/container_root/ws/models/yolo11n-seg.pt')
model_quantized_dynamic = quantize_dynamic(model=model, qconfig_spec={nn.LSTM, nn.Linear}, dtype=torch.qint8, inplace=False) 
model_quantized_dynamic.export(format='onnx')
# model_quantized_dynamic(img_path)

Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.6.0+cpu CPU (AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx)
YOLO11n-seg summary (fused): 265 layers, 2,868,664 parameters, 0 gradients, 10.4 GFLOPs

PyTorch: starting from '/container_root/ws/models/yolo11n-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (5.9 MB)

ONNX: starting export with onnx 1.16.1 opset 19...
ONNX: slimming with onnxslim 0.1.49...
ONNX: export success ✅ 3.7s, saved as '/container_root/ws/models/yolo11n-seg.onnx' (11.2 MB)

Export complete (4.4s)
Results saved to /container_root/ws/models
Predict:         yolo predict task=segment model=/container_root/ws/models/yolo11n-seg.onnx imgsz=640  
Validate:        yolo val task=segment model=/container_root/ws/models/yolo11n-seg.onnx imgsz=640 data=/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


'/container_root/ws/models/yolo11n-seg.onnx'

In [91]:
import onnx
from onnxruntime.transformers import optimizer
from onnxruntime.quantization import quantize_dynamic, QuantType

input_model_path = '/container_root/ws/models/yolo11n-seg.onnx'
mdoel = onnx.load(input_model_path)


model_fp32 = input_model_path
model_quant = '/container_root/ws/models/yolo11n-seg.quant.onnx'
quantized_model = quantize_dynamic(model_fp32, model_quant)

# optimized_model = optimizer.optimize_model(
#   input=input_model_path,
# 	num_heads=0,        # Автоматическое определение параметров
# 	hidden_size=0,
# 	model_type='bert',  # Несмотря на название, работает для CNN
# 	opt_level=99,       # Максимальная оптимизация
# 	optimization_options=optimizer.OptimizationOptions(
# 		enable_gelu=True,
# 		enable_layer_norm=True,
# 		enable_attention=True,
# 		enable_skip_layer_norm=True,
# 		enable_embed_layer_norm=True,
# 		enable_bias_skip_layer_norm=True,
# 		enable_bias_gelu=True
# 	)
# )
